# Example Runthrough of Our Package

The following notebook demonstrates the usage of the functions available when downloading teh `kratosbat` package. We will show how to extract data from the materials project and then further analyze it and predict outcomes using multiple statistical and machine learning methods.

## Extracting data from materialsproject.org
We will first import some relevant modules required for extracting data from the materialsproject.org. The submodule for any data extraction methods is known as `data_extract`:

In [1]:
import data_extract
import pandas as pd

The `get_bat_dat` function is able to mine the entire materials project for all materials classified as battery materials and will return a dataframe of all battery materials and their properties. 

*NOTE* The function takes an API key as an input to access the materialsproject database. We suggest generating your own API key from the dashboard of materials project. 

In [2]:
data_extract.get_bat_dat('EcOxTpa0ymKFe24R')

,Reduced Cell Formula,Average Voltage (V),Min Voltage (V),Max Voltage (V),Number of Steps,Min Instability,Gravimetric Capacity (mAh/g),Volumetric Capacity (Ah/L),Working Ion,Min Fraction,...,Spacegroup,Specific Energy (Wh/kg),Energy Density (Wh/L),Number of Sites,Type,Max Delta Volume,Charge Formula,Discharge Formula,Spacegroup Number,Crystal Lattice
Battery ID,,,,,,,,,,,,,,,,,,,,,
mp-504791_Li,P(WO4)2,2.325185,2.325185,2.325185,1,0.000000,50.228609,269.262771,Li,0.0,...,"{'number': 19, 'hall_number': 115, 'internatio...",116.790816,626.085802,11.0,intercalation,0.028564,P(WO4)2,LiP(WO4)2,19,Orthorombic
mp-763480_Li,P3W2O13,3.291748,3.291748,3.291748,1,0.000097,39.674483,154.215698,Li,0.0,...,"{'number': 14, 'hall_number': 81, 'internation...",130.598387,507.639167,18.0,intercalation,0.017562,P3W2O13,LiP3W2O13,14,Monoclinic
mp-1176966_Li,P8W3O29,3.609671,2.660169,5.062579,3,0.025051,181.943205,651.708695,Li,0.0,...,"{'number': 165, 'hall_number': 457, 'internati...",656.755144,2352.454095,40.0,intercalation,0.061034,P8W3O29,Li2P8W3O29,165,Trigonal
mvc-5592_Li,P2WO7,3.201169,3.201169,3.201169,1,0.049028,73.484217,329.726946,Li,0.0,...,"{'number': 2, 'hall_number': 2, 'international...",235.235410,1055.511733,10.0,intercalation,0.021096,P2WO7,LiP2WO7,2,Triclinic
mp-763566_Li,P2WO7,2.438309,2.438309,2.438309,1,0.000000,73.484217,306.081406,Li,0.0,...,"{'number': 4, 'hall_number': 6, 'international...",179.177232,746.321063,10.0,intercalation,0.025349,P2WO7,LiP2WO7,4,Monoclinic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-1180276_Mg,VCu3Se4,-0.098693,-0.098693,-0.098693,1,0.000000,92.144929,461.340369,Mg,0.0,...,"{'number': 215, 'hall_number': 511, 'internati...",-9.094055,-45.531044,8.0,intercalation,0.069416,VCu3Se4,MgVCu3Se4,215,Cubic
mp-1042619_Mg,Cu3(SnO3)4,1.890890,1.890890,1.890890,1,0.053999,60.789787,387.828816,Mg,0.0,...,"{'number': 204, 'hall_number': 500, 'internati...",114.946775,733.341476,19.0,intercalation,0.006392,Cu3(SnO3)4,MgCu3(SnO3)4,204,Cubic
mp-1046932_Mg,BaTl(NiO3)2,3.071616,3.071616,3.071616,1,0.315336,135.921407,914.229317,Mg,0.0,...,"{'number': 1, 'hall_number': 1, 'international...",417.498404,2808.161640,10.0,intercalation,0.058047,BaTl(NiO3)2,Ba2Mg3Tl2(NiO3)4,1,Triclinic


This function also saves this data as a .csv file called **BatteryData.csv**. This data is also cached in the repository as the `get_bat_dat` function can take over 30 minutes to run as it loops through over 4000 battery materials. Therefore, it may be more convenient to read the given .csv file as shown below.

In [3]:
df = pd.read_csv('BatteryData.csv')
df

,Battery ID,Reduced Cell Formula,Average Voltage (V),Min Voltage (V),Max Voltage (V),Number of Steps,Min Instability,Gravimetric Capacity (mAh/g),Volumetric Capacity (Ah/L),Working Ion,...,Spacegroup,Specific Energy (Wh/kg),Energy Density (Wh/L),Number of Sites,Type,Max Delta Volume,Charge Formula,Discharge Formula,Spacegroup Number,Crystal Lattice
0,mp-504791_Li,P(WO4)2,2.325185,2.325185,2.325185,1,0.000000,50.228609,269.262771,Li,...,"{'number': 19, 'hall_number': 115, 'internatio...",116.790816,626.085802,11.0,intercalation,0.028564,P(WO4)2,LiP(WO4)2,19,Orthorombic
1,mp-763480_Li,P3W2O13,3.291748,3.291748,3.291748,1,0.000097,39.674483,154.215698,Li,...,"{'number': 14, 'hall_number': 81, 'internation...",130.598387,507.639167,18.0,intercalation,0.017562,P3W2O13,LiP3W2O13,14,Monoclinic
2,mp-1176966_Li,P8W3O29,3.609671,2.660169,5.062579,3,0.025051,181.943205,651.708695,Li,...,"{'number': 165, 'hall_number': 457, 'internati...",656.755144,2352.454095,40.0,intercalation,0.061034,P8W3O29,Li2P8W3O29,165,Trigonal
3,mvc-5592_Li,P2WO7,3.201169,3.201169,3.201169,1,0.049028,73.484217,329.726946,Li,...,"{'number': 2, 'hall_number': 2, 'international...",235.235410,1055.511733,10.0,intercalation,0.021096,P2WO7,LiP2WO7,2,Triclinic
4,mp-763566_Li,P2WO7,2.438309,2.438309,2.438309,1,0.000000,73.484217,306.081406,Li,...,"{'number': 4, 'hall_number': 6, 'international...",179.177232,746.321063,10.0,intercalation,0.025349,P2WO7,LiP2WO7,4,Monoclinic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,VCu3Se4,-0.098693,-0.098693,-0.098693,1,0.000000,92.144929,461.340369,Mg,...,"{'number': 215, 'hall_number': 511, 'internati...",-9.094055,-45.531044,8.0,intercalation,0.069416,VCu3Se4,MgVCu3Se4,215,Cubic
4397,mp-1042619_Mg,Cu3(SnO3)4,1.890890,1.890890,1.890890,1,0.053999,60.789787,387.828816,Mg,...,"{'number': 204, 'hall_number': 500, 'internati...",114.946775,733.341476,19.0,intercalation,0.006392,Cu3(SnO3)4,MgCu3(SnO3)4,204,Cubic
4398,mp-1046932_Mg,BaTl(NiO3)2,3.071616,3.071616,3.071616,1,0.315336,135.921407,914.229317,Mg,...,"{'number': 1, 'hall_number': 1, 'international...",417.498404,2808.161640,10.0,intercalation,0.058047,BaTl(NiO3)2,Ba2Mg3Tl2(NiO3)4,1,Triclinic
4399,mp-1041103_Mg,Ho(MoO3)2,1.982244,1.982244,1.982244,1,0.059259,112.348870,735.558063,Mg,...,"{'number': 1, 'hall_number': 1, 'international...",222.702915,1458.055830,9.0,intercalation,0.019274,Ho(MoO3)2,HoMg(MoO3)2,1,Triclinic


We are aware that the materials project is constantly adding materials to its database. To check to see if the cached .csv file is up to date with that available at materialsproject.org, we can simply use the `update_check` function available in the `data_extract` module. Again, this function requires an API key.

In [4]:
data_extract.update_check('EcOxTpa0ymKFe24R')

The Current BatteryData.csv file is up to date!


In [57]:
import predict_tools
import torch
import numpy as np
import pandas as pd

The neural network that we developed utilizes ``PyTorch`` for creation of the model, as well as the forward and backpropogation techniques to properly train the neural network.

We will import our ``predict_tools`` to have all the functions we need henceforth.

Now, we will create a model to predict volumetric and gravimetric capacity based off the information from **materialsproject.org**, normalized using our PCA process. The function takes a dataframe, the number of inputs, the size of the first, then second, hidden layer, and the number of outputs. The final parameter is the number of datapoints that we will use in our model.

In [58]:
model = predict_tools.nn_capacity('NEWMinMaxScaler.csv', 91, 100, 75, 2, 4000)

If we look at our output:

In [59]:
model

Sequential(
  (0): Linear(in_features=91, out_features=100, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=100, out_features=75, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=75, out_features=2, bias=True)
)

We see that it is a PyTorch model! Let's predict some data now.
First, we will create a dataframe that takes some information from the **materialsproject.org** database:

In [60]:
train_bat = pd.read_csv('NEWMinMaxScaler.csv')

x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

x_test = np.array(x_train[4000:])

In [62]:
x_test = torch.tensor(x_test, dtype = torch.float)

In [63]:
predict_tools.get_predictions(model, x_test)

tensor([[0.1988, 0.3182],
        [0.1660, 0.2792],
        [0.2340, 0.3322],
        [0.2080, 0.3126],
        [0.2142, 0.2386],
        [0.2196, 0.2910],
        [0.2073, 0.2613],
        [0.2105, 0.2904],
        [0.2128, 0.2673],
        [0.1963, 0.2645],
        [0.2047, 0.2661],
        [0.1912, 0.2674],
        [0.1881, 0.2714],
        [0.2145, 0.2666],
        [0.1986, 0.2991],
        [0.2185, 0.2627],
        [0.1879, 0.2507],
        [0.1999, 0.2554],
        [0.2038, 0.2302],
        [0.1803, 0.2630],
        [0.1739, 0.2805],
        [0.1958, 0.2827],
        [0.2324, 0.3254],
        [0.1993, 0.2626],
        [0.1854, 0.3011],
        [0.2125, 0.2575],
        [0.1806, 0.2876],
        [0.1917, 0.3066],
        [0.1969, 0.3027],
        [0.1767, 0.2980],
        [0.2189, 0.2601],
        [0.1900, 0.2812],
        [0.2047, 0.2887],
        [0.2108, 0.2340],
        [0.2065, 0.2696],
        [0.2150, 0.2734],
        [0.2031, 0.2828],
        [0.2045, 0.2807],
        [0.2

## For the user

To generate the information needed for the Neural Network, users must use the ``generate_inputs`` Python function:

The user just needs to input their working ion, crystal system number, spacegroup number, charge formula, and discharge formula.

In [8]:
x = predict_tools.generate_model_df('Li', 115, 19, 'P(WO4)2', 'LiP(WO4)2')

/Users/andrewgonsalves/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Then convert their value into a PyTorch ``Tensor``:

In [9]:
x_torch = torch.tensor(x, dtype = torch.float)

Now, calling the model on our data:

In [10]:
y = model(x_torch)

In [11]:
y = model(x_torch).clone().detach()

In [12]:
y = np.array(y)

In [13]:
print(str(y[0]))

[0.1821037  0.08646781]


And that's it!

To scale our data back:

In [14]:
predict_tools.convert(y)

[[469.98715 680.81177]]


As you can see, our gravimetric capcity would be 469.98 and our volumetric capacity is 680!